In [1]:
import pandas as pd
from pathlib import Path

# Cleaned House Listings

### Import house_listing.csv

In [2]:
house_listing_file = Path("resources/house_listings.csv")
house_listing_df = pd.read_csv(house_listing_file,encoding="ISO-8859-1")
house_listing_df.head()

,City,Price,Address,Number_Beds,Number_Baths,Province,Population,Latitude,Longitude,Median_Family_Income
0,Toronto,779900,#318 -20 SOUTHPORT ST,3,2,Ontario,5647656,43.7417,-79.3733,97000.0
1,Toronto,799999,#818 -60 SOUTHPORT ST,3,1,Ontario,5647656,43.7417,-79.3733,97000.0
2,Toronto,799900,#714 -859 THE QUEENSWAY,2,2,Ontario,5647656,43.7417,-79.3733,97000.0
3,Toronto,1200000,275 MORTIMER AVE,4,2,Ontario,5647656,43.7417,-79.3733,97000.0
4,Toronto,668800,#420 -388 RICHMOND ST,1,1,Ontario,5647656,43.7417,-79.3733,97000.0


### Renaming and reordering columns, adjusting data tpyes, formatting address column, removing all listings with zero beds and/or baths

In [3]:
#renaming columns of house_listing_df
house_listing_df = house_listing_df.rename(columns={
    "City":"city",
    "Price":"price",
    "Address":"address",
    "Number_Beds":"beds",
    "Number_Baths":"baths",
    "Province":"province",
    "Population":"population",
    "Latitude":"lat",
    "Longitude":"lon",
    "Median_Family_Income":"median_income"
    })

#reorder the columns
house_listing_df = house_listing_df[[
    "city","province","population","median_income","lat","lon","address","price","beds","baths"
]]

In [4]:
#check data types
house_listing_df.dtypes

city              object
province          object
population         int64
median_income    float64
lat              float64
lon              float64
address           object
price              int64
beds               int64
baths              int64
dtype: object

In [5]:
#change data type of Median_Family_Income from float to int64
house_listing_df = house_listing_df.astype({'median_income':'int64'}, errors='raise')
house_listing_df['median_income'].dtype

dtype('int64')

In [6]:
#clean up the address column by stripping spaces and converting to lowercase
house_listing_df['address'] = house_listing_df['address'].str.strip().str.lower()

#for address column,change any occurence of #NAME? to unknown, regardless of case or extra spaces
house_listing_df['address'] = house_listing_df['address'].str.replace(r'\s*#name\?\s*','unknown', regex=True)

In [7]:
#remove all cases where beds and/or baths are 0
house_listing_df = house_listing_df[(house_listing_df['beds'] > 0) & (house_listing_df['baths'] > 0)]

In [8]:
#reviewing the current state and structure of the DataFrame
house_listing_df.head()

,city,province,population,median_income,lat,lon,address,price,beds,baths
0,Toronto,Ontario,5647656,97000,43.7417,-79.3733,#318 -20 southport st,779900,3,2
1,Toronto,Ontario,5647656,97000,43.7417,-79.3733,#818 -60 southport st,799999,3,1
2,Toronto,Ontario,5647656,97000,43.7417,-79.3733,#714 -859 the queensway,799900,2,2
3,Toronto,Ontario,5647656,97000,43.7417,-79.3733,275 mortimer ave,1200000,4,2
4,Toronto,Ontario,5647656,97000,43.7417,-79.3733,#420 -388 richmond st,668800,1,1


### Validate Data: checking for missing values, nulls, duplicates, and removing duplicates

In [9]:
#there are no incomplete rows
house_listing_df.count()

city             34942
province         34942
population       34942
median_income    34942
lat              34942
lon              34942
address          34942
price            34942
beds             34942
baths            34942
dtype: int64

In [10]:
#check for null values in housing_listing_df
house_listing_df.isnull().values.any()

False

In [11]:
#drop duplicats except the last occurence and keep these changes the source DataFrame house_lisiting_df
house_listing_df.drop_duplicates(keep='last',inplace=True)

# Show the duplicate rows if there are any
if house_listing_df.duplicated(keep=False).any():
    duplicate_rows = house_listing_df[house_listing_df.duplicated(keep=False)]
    print("Duplicate Rows:")
    print(duplicate_rows)
else:
    print("No duplicate rows found.")


No duplicate rows found.


In [12]:
#length of house_listing_df was originally 35768 but once duplicates dropped, length of house_listing_df is 33248
#beds/baths 0 dropped, length of house_listing_df is 33248
len(house_listing_df)

32568

### Export cleaned house_listing_df DataFrame to csv with indext reset

In [13]:
#reset the index of the DataFrame 
house_listing_df = house_listing_df.reset_index(drop=True)

#output a clean version of the csv file
house_listing_df.to_csv("clean_dataset/house_listings_clean.csv", index=False)

### Checking for validity of outliers for House Listings

### Outputting the count of unique bed and bath values in dataset

In [14]:
#there are house listings with beds of more than 10
house_listing_df["beds"].unique()

array([  3,   2,   4,   1,   6,   5,   7,   8,  11,   9,  10,  15,  16,
        12,  40,  30,  20,  36,  14,  17,  46, 109,  35,  21,  18,  47,
        27,  26,  13,  19], dtype=int64)

In [15]:
#there are house listings with bathrooms of more than 10
house_listing_df["baths"].unique()

array([ 2,  1,  3,  4,  5,  6,  7,  8,  9, 11, 10, 12, 16, 25, 17, 59, 15,
       21, 14, 20, 22], dtype=int64)

### Filtered house listings with less than 10 beds and/or baths

In [16]:
#clean the data and remove all listings with more than 10 beds or baths
housing_filtered = house_listing_df[(house_listing_df["beds"] <= 10) & (house_listing_df["baths"] <= 10)]

In [17]:
#number of beds after cleaning 
housing_filtered['beds'].unique()

array([ 3,  2,  4,  1,  6,  5,  7,  8,  9, 10], dtype=int64)

In [18]:
#number of baths after cleaning
housing_filtered['baths'].unique()

array([ 2,  1,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int64)

### Validate Data: checking for missing values, nulls and duplicates

In [19]:
housing_filtered.count()

city             32497
province         32497
population       32497
median_income    32497
lat              32497
lon              32497
address          32497
price            32497
beds             32497
baths            32497
dtype: int64

In [20]:
#check for null values in population_dwelling_df
housing_filtered.isnull().values.any()

False

In [21]:
housing_filtered.head()

,city,province,population,median_income,lat,lon,address,price,beds,baths
0,Toronto,Ontario,5647656,97000,43.7417,-79.3733,#318 -20 southport st,779900,3,2
1,Toronto,Ontario,5647656,97000,43.7417,-79.3733,#818 -60 southport st,799999,3,1
2,Toronto,Ontario,5647656,97000,43.7417,-79.3733,#714 -859 the queensway,799900,2,2
3,Toronto,Ontario,5647656,97000,43.7417,-79.3733,275 mortimer ave,1200000,4,2
4,Toronto,Ontario,5647656,97000,43.7417,-79.3733,#420 -388 richmond st,668800,1,1


In [22]:
# Check for duplicates
duplicates = housing_filtered.duplicated()

# Display the duplicate rows
duplicate_rows = housing_filtered[duplicates]

# Show the duplicate rows if there are any
if not duplicate_rows.empty:
    print("Duplicate Rows:")
    print(duplicate_rows)
else:
    print("No duplicate rows found.")


No duplicate rows found.


### Evaluating the impact of outliers: A comparison of mean, median, standard deviation and quartiles

In [23]:
#description of DataFrame with outliers
statistical_summary = house_listing_df['price'].describe()
formatted = statistical_summary.apply(lambda x: f'{x:,.2f}')
print(formatted)

count        32,568.00
mean        983,653.74
std       1,041,230.06
min          21,500.00
25%         499,376.75
50%         729,000.00
75%       1,129,900.00
max      37,000,000.00
Name: price, dtype: object


In [24]:
#description of DataFrame without outliers of more than 10 beds and/or 10 baths
statistical_summary_filtered = housing_filtered['price'].describe()
formatted_filter = statistical_summary_filtered.apply(lambda x: f'{x:,.2f}')
print(formatted_filter)

count        32,497.00
mean        977,390.30
std       1,007,043.03
min          21,500.00
25%         499,000.00
50%         729,000.00
75%       1,125,000.00
max      28,800,000.00
Name: price, dtype: object


Based on the descriptions of the house_listing_df (with the possible outliers) and the housing_filtered (without the possible outliers), the mean shows a slight decrease but the median remains the same. The changes in the standard deviation and quartlies show that outliers have a minor impact on the overall dataset.

In [25]:
#reset the index of the DataFrame 
#housing_filtered = housing_filtered.reset_index(drop=True)
#output a clean version of the csv file
#housing_filtered.to_csv("clean_dataset/housing_filtered_clean.csv", index=False)

# Cleaned Population Dwelling Counts

### Import population_dwelling_counts.csv

In [26]:
population_dwelling_file = Path("resources/population_dwelling_counts.csv")
population_dwelling_df = pd.read_csv(population_dwelling_file,encoding='ISO-8859-1')
population_dwelling_df.head()

,ï»¿Geographic name,Geographic area type,Province and Territory Abbrev,"Population, 2021","Total private dwellings, 2021","Private dwellings occupied by usual residents, 2021","Land area in square kilometres, 2021","Population density per square kilometre, 2021"
0,Canada 2,Country,...,"36,991,981","16,284,235","14,978,941","8,788,702.80",4.2
1,Newfoundland and Labrador,Province,N.L.,"510,550","269,184","223,253","358,170.37",1.4
2,Bay Roberts,Small population centre,N.L.,"6,897","3,184","2,888",16.88,408.5
3,Bishop's Falls,Small population centre,N.L.,"3,006","1,498","1,386",6.89,436.1
4,Bonavista,Small population centre,N.L.,"2,979","1,628","1,379",4.36,683.1


### Deleting a column, renaming and reordering columns, filling missing data, formatting abbreviations column

In [27]:
#column named, Private dwellings occupied by usual residents, 2021 includes renter and owners who live in the property
#column named, Total private dwellings, 2021 includes renters and owners who live in the property plus short term rentals such as cottages and airbnbs
#only need column named, Total private dwellings, 2021  
del population_dwelling_df["Private dwellings occupied by usual residents, 2021"]

In [28]:
#change population_dwelling_df column names
population_dwelling_df = population_dwelling_df.rename(columns={
    "ï»¿Geographic name":"city",
    "Geographic area type":"area_type",
    "Province and Territory Abbrev":"abbrev",
    "Population, 2021":"pop",
    "Total private dwellings, 2021":"total_dwellings",
    "Land area in square kilometres, 2021":"land_area_sq_km",
    "Population density per square kilometre, 2021":"pop_density_sq_km"
    })

#reorder the columns
population_dwelling_df = population_dwelling_df[[
    "city","abbrev","area_type","pop","pop_density_sq_km","total_dwellings","land_area_sq_km"   
]]

population_dwelling_df.head()

,city,abbrev,area_type,pop,pop_density_sq_km,total_dwellings,land_area_sq_km
0,Canada 2,...,Country,"36,991,981",4.2,"16,284,235","8,788,702.80"
1,Newfoundland and Labrador,N.L.,Province,"510,550",1.4,"269,184","358,170.37"
2,Bay Roberts,N.L.,Small population centre,"6,897",408.5,"3,184",16.88
3,Bishop's Falls,N.L.,Small population centre,"3,006",436.1,"1,498",6.89
4,Bonavista,N.L.,Small population centre,"2,979",683.1,"1,628",4.36


In [29]:
#add abbreviation for Canada
population_dwelling_df.loc[population_dwelling_df["city"] == "Canada 2","abbrev"] = "CAN"

#remove periods from the abbreviations column
population_dwelling_df['abbrev'] = population_dwelling_df['abbrev'].str.replace(".","")

population_dwelling_df.head()

,city,abbrev,area_type,pop,pop_density_sq_km,total_dwellings,land_area_sq_km
0,Canada 2,CAN,Country,"36,991,981",4.2,"16,284,235","8,788,702.80"
1,Newfoundland and Labrador,NL,Province,"510,550",1.4,"269,184","358,170.37"
2,Bay Roberts,NL,Small population centre,"6,897",408.5,"3,184",16.88
3,Bishop's Falls,NL,Small population centre,"3,006",436.1,"1,498",6.89
4,Bonavista,NL,Small population centre,"2,979",683.1,"1,628",4.36


### Simplifying area type labels, formatting numerical data, adjusting data types

In [30]:
population_dwelling_df["area_type"].value_counts()

area_type
Small population centre          938
Medium population centre          60
Large urban population centre     36
Province                          10
Territory                          3
Country                            1
Name: count, dtype: int64

In [31]:
#simplifying area type labels
population_dwelling_df["area_type"] = population_dwelling_df["area_type"].replace({
    "Small population centre":"small",
    "Medium population centre":"medium",
    "Large urban population centre":"large urban"
})
population_dwelling_df["area_type"].value_counts()

area_type
small          938
medium          60
large urban     36
Province        10
Territory        3
Country          1
Name: count, dtype: int64

In [32]:
#check data types
population_dwelling_df.dtypes

city                 object
abbrev               object
area_type            object
pop                  object
pop_density_sq_km    object
total_dwellings      object
land_area_sq_km      object
dtype: object

In [33]:
#creating a function to remove all commas from numerical data
def remove_comma(column):
    return column.astype(str).str.replace(',','')

population_dwelling_df['pop'] = remove_comma(population_dwelling_df['pop'])
population_dwelling_df['pop_density_sq_km'] = remove_comma(population_dwelling_df['pop_density_sq_km'])
population_dwelling_df['total_dwellings'] = remove_comma(population_dwelling_df['total_dwellings'])
population_dwelling_df['land_area_sq_km'] = remove_comma(population_dwelling_df['land_area_sq_km'])

#using df.astype() method to convert the data types of the numerical columns
population_dwelling_df = population_dwelling_df.astype({
    'pop':'int64',
    'pop_density_sq_km':'float64',
    'total_dwellings':'int64',
    'land_area_sq_km':'float64'
}, errors='raise')

#check the data types of the columns to confirm the conversion
print(population_dwelling_df.dtypes)

city                  object
abbrev                object
area_type             object
pop                    int64
pop_density_sq_km    float64
total_dwellings        int64
land_area_sq_km      float64
dtype: object


### Checking for missing values, nulls, duplicates, and removing duplicates

In [34]:
# Check for duplicates
duplicates = population_dwelling_df.duplicated()

# Display the duplicate rows
duplicate_rows = population_dwelling_df[duplicates]

# Show the duplicate rows if there are any
if not duplicate_rows.empty:
    print("Duplicate Rows:")
    print(duplicate_rows)
else:
    print("No duplicate rows found.")


No duplicate rows found.


In [35]:
#there are no incomplete rows
population_dwelling_df.count()

city                 1048
abbrev               1048
area_type            1048
pop                  1048
pop_density_sq_km    1048
total_dwellings      1048
land_area_sq_km      1048
dtype: int64

In [36]:
#check for null values
population_dwelling_df.isnull().values.any()

False

### Removing rows by area type, creating a new DataFrame called province_territory_population_dwelling_df

In [37]:
#checking length of population_dwelling_df before removing rows where area_type is Province, Territory and Country
len(population_dwelling_df)

1048

In [38]:
#create new DataFrame for rows where area_type is Province, Territory and Country
province_territory_population_dwelling_df = population_dwelling_df[
    (population_dwelling_df["area_type"] == "Province") | 
    (population_dwelling_df["area_type"] == "Country") | 
    (population_dwelling_df["area_type"] == "Territory")
]
#remove all rows from population_dwelling_df, where area_type is Province, Territory and Country
population_dwelling_df = population_dwelling_df.drop(province_territory_population_dwelling_df.index)

#reset the index of the DataFrame 
province_territory_population_dwelling_df = province_territory_population_dwelling_df.reset_index(drop=True)
province_territory_population_dwelling_df

,city,abbrev,area_type,pop,pop_density_sq_km,total_dwellings,land_area_sq_km
0,Canada 2,CAN,Country,36991981,4.2,16284235,8788702.80
1,Newfoundland and Labrador,NL,Province,510550,1.4,269184,358170.37
2,Prince Edward Island,PEI,Province,154331,27.2,74934,5681.18
3,Nova Scotia,NS,Province,969383,18.4,476007,52824.71
4,New Brunswick,NB,Province,775610,10.9,366146,71248.50
5,Quebec 2,Que,Province,8501833,6.5,4050164,1298599.75
6,Ontario 2,Ont,Province,14223942,15.9,5929250,892411.76
7,Manitoba 2,Man,Province,1342153,2.5,571528,540310.19
8,Saskatchewan 2,Sask,Province,1132505,2.0,513725,577060.40
9,Alberta 2,Alta,Province,4262635,6.7,1772670,634658.27


In [39]:
#checking length of new DataFrame province_territory_population_dwelling_df 
len(province_territory_population_dwelling_df)

14

In [40]:
#checking length of population_dwelling_df after removing rows where area_type is Province, Territory and Country
len(population_dwelling_df)

1034

### Checking for missing values, nulls, and data type in new DataFrame

In [41]:
#there are no incomplete rows
province_territory_population_dwelling_df.count()

city                 14
abbrev               14
area_type            14
pop                  14
pop_density_sq_km    14
total_dwellings      14
land_area_sq_km      14
dtype: int64

In [42]:
#check for null values
population_dwelling_df.isnull().values.any()

False

In [43]:
#checking data types of province_territory_population_dwelling_df
province_territory_population_dwelling_df.dtypes

city                  object
abbrev                object
area_type             object
pop                    int64
pop_density_sq_km    float64
total_dwellings        int64
land_area_sq_km      float64
dtype: object

### Export province_territory_population_dwelling_df DataFrame to csv with index reset

In [44]:
#reset the index of the DataFrame 
province_territory_population_dwelling_df = province_territory_population_dwelling_df.reset_index(drop=True)

#output a clean version of the csv file
province_territory_population_dwelling_df.to_csv("clean_dataset/province_territory_population_dwelling_clean.csv", index=False)

### Double check the population_dwelling_df for incomplete rows

In [45]:
#double check, there are no incomplete rows
population_dwelling_df.count()

city                 1034
abbrev               1034
area_type            1034
pop                  1034
pop_density_sq_km    1034
total_dwellings      1034
land_area_sq_km      1034
dtype: int64

### Export cleaned, population_dwelling_df DataFrame to csv with index reset

In [46]:
#reset the index of the DataFrame 
population_dwelling_df = population_dwelling_df.reset_index(drop=True)

#output a clean version of the csv file
population_dwelling_df.to_csv("clean_dataset/population_dwelling_clean.csv", index=False)

# Merged House Lisiting and Population Dwelling on Cities

In [47]:
population_housing_merged = population_dwelling_df.merge(house_listing_df, on='city', how="inner")
population_housing_merged.head()

,city,abbrev,area_type,pop,pop_density_sq_km,total_dwellings,land_area_sq_km,province,population,median_income,lat,lon,address,price,beds,baths
0,St. John's,NL,large urban,185565,1042.5,86189,178.0,Newfoundland and Labrador,185565,85000,47.4817,-52.7971,33 queens road,275000,4,1
1,St. John's,NL,large urban,185565,1042.5,86189,178.0,Newfoundland and Labrador,185565,85000,47.4817,-52.7971,70 julieann place,767500,4,4
2,St. John's,NL,large urban,185565,1042.5,86189,178.0,Newfoundland and Labrador,185565,85000,47.4817,-52.7971,5 ruth avenue,159900,3,1
3,St. John's,NL,large urban,185565,1042.5,86189,178.0,Newfoundland and Labrador,185565,85000,47.4817,-52.7971,11 parliament street,350000,4,3
4,St. John's,NL,large urban,185565,1042.5,86189,178.0,Newfoundland and Labrador,185565,85000,47.4817,-52.7971,84 gil eannes drive,424900,4,3


### Deleting, renaming, and reordering columns

In [48]:
#delete population column since we have pop column
del population_housing_merged['population']

In [49]:
#reorder the columns
population_housing_merged = population_housing_merged[[
    "city","province","abbrev","area_type","pop","pop_density_sq_km","total_dwellings","land_area_sq_km","median_income",
    "lat","lon","address","price","beds","baths",
]]
population_housing_merged.head()

,city,province,abbrev,area_type,pop,pop_density_sq_km,total_dwellings,land_area_sq_km,median_income,lat,lon,address,price,beds,baths
0,St. John's,Newfoundland and Labrador,NL,large urban,185565,1042.5,86189,178.0,85000,47.4817,-52.7971,33 queens road,275000,4,1
1,St. John's,Newfoundland and Labrador,NL,large urban,185565,1042.5,86189,178.0,85000,47.4817,-52.7971,70 julieann place,767500,4,4
2,St. John's,Newfoundland and Labrador,NL,large urban,185565,1042.5,86189,178.0,85000,47.4817,-52.7971,5 ruth avenue,159900,3,1
3,St. John's,Newfoundland and Labrador,NL,large urban,185565,1042.5,86189,178.0,85000,47.4817,-52.7971,11 parliament street,350000,4,3
4,St. John's,Newfoundland and Labrador,NL,large urban,185565,1042.5,86189,178.0,85000,47.4817,-52.7971,84 gil eannes drive,424900,4,3


### Checking how many cities are missing in merged DataFrame

In [50]:
#population_dwelling dataframe has 1035 different cities
population_dwelling_df['city'].nunique()

1022

In [51]:
#house_listing dataframe has 45 different cities
house_listing_df['city'].nunique()

45

In [52]:
#newly created population_housing_merged has 42 different cities
population_housing_merged['city'].nunique()

42

In [53]:
#see what is cities are missing
#get the unique city names from both DataFrames
housing_cities = set(house_listing_df["city"].unique())
merged_cities = set(population_housing_merged['city'].unique())

#find the cities in housing_df that are missing in cleaned_df
missing_cities = housing_cities - merged_cities 

#to see the result
print(missing_cities)

{'Maple Ridge', 'Burnaby', 'New Westminster'}


### Checking for missing values, nulls, data type and duplicates in merged DataFrame

In [54]:
#there are no incomplete rows
population_housing_merged.count()

city                 30143
province             30143
abbrev               30143
area_type            30143
pop                  30143
pop_density_sq_km    30143
total_dwellings      30143
land_area_sq_km      30143
median_income        30143
lat                  30143
lon                  30143
address              30143
price                30143
beds                 30143
baths                30143
dtype: int64

In [55]:
#check for null values in population_housing_merged
population_housing_merged.isnull().values.any()

False

In [56]:
#check data types
population_housing_merged.dtypes

city                  object
province              object
abbrev                object
area_type             object
pop                    int64
pop_density_sq_km    float64
total_dwellings        int64
land_area_sq_km      float64
median_income          int64
lat                  float64
lon                  float64
address               object
price                  int64
beds                   int64
baths                  int64
dtype: object

In [57]:
# Check for duplicates
duplicates = population_housing_merged.duplicated()

# Display the duplicate rows
duplicate_rows = population_housing_merged[duplicates]

# Show the duplicate rows if there are any
if not duplicate_rows.empty:
    print("Duplicate Rows:")
    print(duplicate_rows)
else:
    print("No duplicate rows found.")


No duplicate rows found.


### Export population_housing_merged DataFrame to csv with reset index

In [58]:
#reset the index of the DataFrame 
population_housing_merged = population_housing_merged.reset_index(drop=True)

In [59]:
#output a clean version of the csv file
population_housing_merged.to_csv("clean_dataset/population_housing_merged_clean.csv", index=False)